# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Рекуррентные нейронные сети

### Оценивание и штрафы


Максимально допустимая оценка за работу — __10 (+3) баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 5.10.25 23:59__   
__Жесткий дедлайн: 8.10.25 23:59__


### О задании

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pandas as pd

dataset = pd.read_csv('data/biotech_news.tsv', sep='\t')
dataset.head()

,text,labels
0,drive your plow over the bones of the dead by ...,other
1,in the recently tabled national budget denel h...,other
2,shares take a break its good for you picture g...,other
3,reso is currently hiring for two positions pro...,other
4,charter buyer club what is the charter buyer c...,other


## Предобработка лейблов


__Задание 1 (0.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [2]:
res = set()
for s in dataset.labels:
    labels = s.split(', ')
    res = res | set(labels)
keys = sorted(list(res))
labels = dict(zip(keys, range(len(keys))))
labels

{'alliance & partnership': 0,
 'article publication': 1,
 'clinical trial sponsorship': 2,
 'closing': 3,
 'company description': 4,
 'department establishment': 5,
 'event organization': 6,
 'executive appointment': 7,
 'executive statement': 8,
 'expanding geography': 9,
 'expanding industry': 10,
 'foundation': 11,
 'funding round': 12,
 'hiring': 13,
 'investment in public company': 14,
 'ipo exit': 15,
 'm&a': 16,
 'new initiatives & programs': 17,
 'new initiatives or programs': 18,
 'other': 19,
 'participation in an event': 20,
 'partnerships & alliances': 21,
 'patent publication': 22,
 'product launching & presentation': 23,
 'product updates': 24,
 'regulatory approval': 25,
 'service & product providing': 26,
 'subsidiary establishment': 27,
 'support & philanthropy': 28}

In [3]:
import numpy as np
def encode_labels(labels_str: str):
    res = np.zeros(len(labels))
    for l in labels_str.split(', '):
        res[labels[l]] = 1
    return res
encode_labels('alliance & partnership, closing')

array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [4]:
dataset['encoded_labels'] = dataset.labels.apply(encode_labels)
dataset

,text,labels,encoded_labels
0,drive your plow over the bones of the dead by ...,other,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,in the recently tabled national budget denel h...,other,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,shares take a break its good for you picture g...,other,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,reso is currently hiring for two positions pro...,other,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,charter buyer club what is the charter buyer c...,other,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
3034,published less than an hour ago a grateful fam...,"funding round, support & philanthropy, executi...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3035,a cenexelcenter of excellence joined nearly 10...,clinical trial sponsorship,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3036,jun 29 2020 8 47 a m pt reply in response to t...,"new initiatives or programs, funding round, ex...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ..."
3037,whatsapp photo supplied red river waste soluti...,"service & product providing, closing, company ...","[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [21]:
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(
    dataset.text.values,
    dataset.encoded_labels.values,
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

__Задание 2 (1 балл)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

СДЕЛАНО В dataset.py

__Задание 3 (1.5 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +3 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

In [23]:
from vocab import Vocab
from dataset import TextDataset, collate

from torch.utils.data import DataLoader

vocab = Vocab(texts_train)

train_dataset = TextDataset(texts_train, y_train, vocab)
test_dataset = TextDataset(texts_test, y_test, vocab)

train_loader = DataLoader(
    train_dataset, 
    batch_size=32, 
    shuffle=True,
    collate_fn=collate
)

val_loader = DataLoader(
    test_dataset, 
    batch_size=128, 
    shuffle=False,
    collate_fn=collate
)

In [ ]:
for batch_texts, batch_labels in train_loader:
    print(f"Texts shape: {batch_texts.shape}")
    print(f"Labels shape: {batch_labels.shape}")
    print(f"Labels dtype: {batch_labels.dtype}")
    break

Texts shape: torch.Size([32, 647])
Labels shape: torch.Size([32, 29])
Labels dtype: torch.float32


## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [ ]:
from sklearn.metrics import f1_score

def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape

    return f1_score(y_true, y_pred, average='macro')

## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс. 

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [24]:
from rnn import RNN

In [25]:
from train import train

In [ ]:
import torch
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss

device = 'cpu'

class_weights = torch.tensor((y_train.shape[0] - y_train.sum(0)) / (y_train.sum(0) + 1e-6), dtype=torch.float32, device=device)

model = RNN(
    vocab_size=len(vocab.dictionary),
    embed_dim=32,
    hidden_size=64,
    num_classes=29
)

optim = Adam(model.parameters(), lr=1e-3)
criterion = BCEWithLogitsLoss(pos_weight=class_weights)

gt = torch.stack([torch.tensor(l) for l in y_test], dim=0).cpu()
train(5, model=model, optim=optim, criterion=criterion, train_loader=train_loader, val_loader=val_loader, val_gt=gt)

Epoch 0: Train Loss: 0.4040, Val Loss: 0.2838


### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\ 
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

In [ ]:
# your code here

__Задание 6 (2 балла).__ Главный недостаток RNN моделей заключается в том, что при сжатии всей информации в один вектор, важные детали пропадают. Для решения этой проблемы был придуман механизм внимания. Реализуйте его по [оригинальной статье](https://arxiv.org/abs/1409.0473). Замерьте качество и сделайте выводы.   
Обратите внимание, что метод был предложен для Encoder-Decoder моделей. В нашем случае декодера нет, поэтому встройте внимание в энкодер: каждый блок LSTM будет смотреть на выходы всех предыдущих блоков.   

In [ ]:
# your code here

__Задание 7 (1 балл).__ Добавьте в вашу реализации возможность увеличивать число слоев LSTM. Обучите модель с двумя слоями и замерьте качество. Сделайте выводы: стоит ли увеличивать размер модели?

In [ ]:
# your code here